# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil.

It's no small feat that most of these ingredients are readily available in grocery stores. In this project, you'll conduct a supply chain analysis of the ingredients used in an avocado toast, utilizing the [Open Food Facts database](https://world.openfoodfacts.org/). This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish. The data is contained in `.csv` files in the `data/` folder provided.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

![](avocado_wallpaper.jpeg)

In [4]:

import pandas as pd

avocado = pd.read_csv('data/avocado.csv', delimiter='\t')
avocado


,code,lc,product_name_de,product_name_el,product_name_en,product_name_es,product_name_fi,product_name_fr,product_name_id,product_name_it,product_name_lt,product_name_lv,product_name_nb,product_name_nl,product_name_pl,product_name_ro,product_name_sl,product_name_sv,generic_name_de,generic_name_en,generic_name_es,generic_name_fr,generic_name_sv,quantity,serving_size,packaging,packaging_tags,brands,brands_tags,brand_owner,categories,categories_tags,labels,labels_tags,countries,countries_tags,stores,stores_tags,obsolete,obsolete_since_date,...,phosphorus_unit,iron_value,iron_unit,magnesium_value,magnesium_unit,zinc_value,zinc_unit,copper_value,copper_unit,manganese_value,manganese_unit,selenium_value,selenium_unit,fruits-vegetables-nuts_value,fruits-vegetables-nuts_unit,phylloquinone_value,phylloquinone_unit,link,off:food_groups,off:food_groups_tags,off:nova_groups,off:nova_groups_tags,off:nutriscore_grade,off:nutriscore_score,off:ecoscore_grade,off:ecoscore_score,off:ecoscore_data.missing_key_data,off:ecoscore_data.agribalyse.code,off:ecoscore_data.adjustments.origins_of_ingredients.value,off:ecoscore_data.adjustments.packaging.value,off:ecoscore_data.adjustments.packaging.non_recyclable_and_non_biodegradable_materials,off:ecoscore_data.adjustments.production_system.value,off:ecoscore_data.adjustments.threatened_species.value,sources_fields:org-database-usda:available_date,sources_fields:org-database-usda:fdc_category,sources_fields:org-database-usda:fdc_data_source,sources_fields:org-database-usda:fdc_id,sources_fields:org-database-usda:modified_date,sources_fields:org-database-usda:publication_date,data_sources
0,0059749979702,fr,NaN,NaN,NaN,NaN,NaN,Naturalia Avocado Oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Naturalia,naturalia,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,Canada,en:canada,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fats,"en:fats-and-sauces,en:fats",2.0,en:2-processed-culinary-ingredients,b,1.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps"
1,7610095131409,en,NaN,NaN,NaN,NaN,NaN,Avocado Bowl chips,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zweifel,zweifel,NaN,"Snacks, Salty snacks, Appetizers, Chips and fr...","en:snacks,en:salty-snacks,en:appetizers,en:chi...","Vegetarian, Vegan","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN,1,2023/03/21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:appetizers,"en:salty-snacks,en:appetizers",NaN,unknown,c,8.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps, Producers, Producer - zweifel"
2,4005514005578,en,NaN,NaN,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tartex,tartex,NaN,de:Abendbrotsufstrich,de:abendbrotsufstrich,"Organic, EU Organic, EG-Öko-Verordnung","en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,b,0.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps, App - smoothie-openfoodfacts"
3,0879890002513,en,NaN,NaN,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,en:4-ultra-processed-food-and-drink-products,NaN,NaN,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps, App - InFood"
4,0223086613685,en,NaN,NaN,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-stat

In [3]:
subset= ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']
avocado = avocado[subset]
avocado

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,0819573012712,en,"Organic Baby Food, Apples, Kale & Avocados",NaN,113 g (113 g),NaN,Happybaby,happybaby,NaN,NaN,United States,en:united-states,NaN,NaN
1781,0052200072097,en,"Just Pineapple, Pear & Avocado",NaN,60 g (60 g),NaN,"Beech-Nut, Beech-Nut Nutrition Company","beech-nut,beech-nut-nutrition-company",NaN,NaN,United States,en:united-states,NaN,NaN
1782,0793613300000,en,Spinach Avocado Dip,NaN,28 g (2 Tbsp),NaN,Classy Delites,classy-delites,NaN,NaN,United States,en:united-states,NaN,NaN
1783,05252428,en,"Organic Just Apple, Raspberry & Avocado, Apple...",NaN,60 g (0.25 cup),NaN,Beech-Nut,beech-nut,NaN,NaN,United States,en:united-states,NaN,NaN


In [11]:

avocado = avocado.dropna(subset='categories_tags')  # Drop NaN values
avocado['categories_list'] = avocado['categories_tags'].str.split(',')
avocado[:2]


,code,lc,product_name_de,product_name_el,product_name_en,product_name_es,product_name_fi,product_name_fr,product_name_id,product_name_it,product_name_lt,product_name_lv,product_name_nb,product_name_nl,product_name_pl,product_name_ro,product_name_sl,product_name_sv,generic_name_de,generic_name_en,generic_name_es,generic_name_fr,generic_name_sv,quantity,serving_size,packaging,packaging_tags,brands,brands_tags,brand_owner,categories,categories_tags,labels,labels_tags,countries,countries_tags,stores,stores_tags,obsolete,obsolete_since_date,...,iron_value,iron_unit,magnesium_value,magnesium_unit,zinc_value,zinc_unit,copper_value,copper_unit,manganese_value,manganese_unit,selenium_value,selenium_unit,fruits-vegetables-nuts_value,fruits-vegetables-nuts_unit,phylloquinone_value,phylloquinone_unit,link,off:food_groups,off:food_groups_tags,off:nova_groups,off:nova_groups_tags,off:nutriscore_grade,off:nutriscore_score,off:ecoscore_grade,off:ecoscore_score,off:ecoscore_data.missing_key_data,off:ecoscore_data.agribalyse.code,off:ecoscore_data.adjustments.origins_of_ingredients.value,off:ecoscore_data.adjustments.packaging.value,off:ecoscore_data.adjustments.packaging.non_recyclable_and_non_biodegradable_materials,off:ecoscore_data.adjustments.production_system.value,off:ecoscore_data.adjustments.threatened_species.value,sources_fields:org-database-usda:available_date,sources_fields:org-database-usda:fdc_category,sources_fields:org-database-usda:fdc_data_source,sources_fields:org-database-usda:fdc_id,sources_fields:org-database-usda:modified_date,sources_fields:org-database-usda:publication_date,data_sources,categories_list
5,3662994002063,fr,NaN,NaN,NaN,NaN,NaN,Avocat hass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 fruits,NaN,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,France,en:france,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fruits,"en:fruits-and-vegetables,en:fruits",NaN,unknown,NaN,NaN,b,66.0,NaN,13004.0,-5.0,-1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - smoothie-openfoodfacts, Apps, App - Yuka","[en:plant-based-foods-and-beverages, en:plant-..."
6,8437013031011,fr,NaN,NaN,NaN,NaN,NaN,Avocat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 kg,NaN,NaN,NaN,NaN,NaN,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,France,en:france,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fruits,"en:fruits-and-vegetables,en:fruits",NaN,unknown,NaN,NaN,c,52.0,1.0,13004.0,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps","[en:plant-based-foods-and-beverages, en:plant-..."


In [12]:

relevent_cat_tags_list=['en:avocadoes', 'en:avocados', 'en:fresh-foods', 'en:fresh-vegetables', 'en:fruchte', 'en:fruits', 'en:raw-green-avocados', 'en:tropical-fruits', 'en:tropische-fruchte', 'en:vegetables-based-foods','fr:hass-avocados']

avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevent_cat_tags_list]))]
avocado

,code,lc,product_name_de,product_name_el,product_name_en,product_name_es,product_name_fi,product_name_fr,product_name_id,product_name_it,product_name_lt,product_name_lv,product_name_nb,product_name_nl,product_name_pl,product_name_ro,product_name_sl,product_name_sv,generic_name_de,generic_name_en,generic_name_es,generic_name_fr,generic_name_sv,quantity,serving_size,packaging,packaging_tags,brands,brands_tags,brand_owner,categories,categories_tags,labels,labels_tags,countries,countries_tags,stores,stores_tags,obsolete,obsolete_since_date,...,iron_value,iron_unit,magnesium_value,magnesium_unit,zinc_value,zinc_unit,copper_value,copper_unit,manganese_value,manganese_unit,selenium_value,selenium_unit,fruits-vegetables-nuts_value,fruits-vegetables-nuts_unit,phylloquinone_value,phylloquinone_unit,link,off:food_groups,off:food_groups_tags,off:nova_groups,off:nova_groups_tags,off:nutriscore_grade,off:nutriscore_score,off:ecoscore_grade,off:ecoscore_score,off:ecoscore_data.missing_key_data,off:ecoscore_data.agribalyse.code,off:ecoscore_data.adjustments.origins_of_ingredients.value,off:ecoscore_data.adjustments.packaging.value,off:ecoscore_data.adjustments.packaging.non_recyclable_and_non_biodegradable_materials,off:ecoscore_data.adjustments.production_system.value,off:ecoscore_data.adjustments.threatened_species.value,sources_fields:org-database-usda:available_date,sources_fields:org-database-usda:fdc_category,sources_fields:org-database-usda:fdc_data_source,sources_fields:org-database-usda:fdc_id,sources_fields:org-database-usda:modified_date,sources_fields:org-database-usda:publication_date,data_sources,categories_list
5,3662994002063,fr,NaN,NaN,NaN,NaN,NaN,Avocat hass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 fruits,NaN,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,France,en:france,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fruits,"en:fruits-and-vegetables,en:fruits",NaN,unknown,NaN,NaN,b,66.0,NaN,13004.0,-5.0,-1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - smoothie-openfoodfacts, Apps, App - Yuka","[en:plant-based-foods-and-beverages, en:plant-..."
6,8437013031011,fr,NaN,NaN,NaN,NaN,NaN,Avocat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 kg,NaN,NaN,NaN,NaN,NaN,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,France,en:france,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fruits,"en:fruits-and-vegetables,en:fruits",NaN,unknown,NaN,NaN,c,52.0,1.0,13004.0,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps","[en:plant-based-foods-and-beverages, en:plant-..."
14,4016249238155,de,Saatenliebe Mohn Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135g,100g,Gläschen,de:gläschen,Allos,allos,NaN,"Pflanzliche Lebensmittel und Getränke, Pflanzl...","en:plant-based-foods-and-beverages,en:plant-ba...","Bio, Vegetarisch, EU-Öko-Verordnung, Glutenfre...","en:organic,en:vegetarian,en:eu-organic,en:no-g...",Deutschland,en:germany,"Bioladen,Alnatura","bioladen,alnatura",0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:salty-and-fatty-products,"en:salty-snacks,en:salty-and-fatty-products",3.0,en:3-processed-foods,b,2.0,b,67.0,1.0,13004.0,-5.0,-15.0,1.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps","[en:plant-based-foods-and-beverages, en:plant-..."
17,8718963381532,de,EAT ME AVOCADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pflanzliche Lebensmittel und Getränke, Pflanzl...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,Deutschland,en:germany,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fruits,"en:fruits-and-vegetables,en

In [13]:

avocado_origin = avocado[avocado['countries'] == 'United Kingdom']
count= avocado_origin['origins_tags'].value_counts()
print("-----Avocado Origin-----")
print(count) # peru=2 is highest among all origins_tags
avocado_origin= 'Peru'



# user-defined function to read and filter data
def read_and_filter_data(filepath, relevant_categories):
  df = pd.read_csv('data/' + filepath, sep='\t')

  df = df[subset]

  df['categories_list'] = df['categories_tags'].str.split(',')

  df = df.dropna(subset = 'categories_list')

  df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]
    
  df = df[(df['countries']=='United Kingdom')]
  print(f'-----{filepath[:-4]} Origins-----','\n',df['origins_tags'].value_counts(), '\n')

  return df



# Identify relevant categories for lemons
relevant_lemon_categories = [
 'en:aromatic-herbs',
 'en:aromatic-plants',
 'en:citron',
 'en:citrus',
 'en:fresh-fruits',
 'en:fresh-lemons',
 'en:fruits',
 'en:lemons',
 'en:unwaxed-lemons'
]

#user-defined function on lemon.csv
lemon = read_and_filter_data('lemon.csv',relevant_lemon_categories)
lemon_origin = 'South Africa' # South Africa=1 is highest among all origins_tags


#user-defined function on olive_oil.csv
with open("data/relevant_olive_oil_categories.txt", "r") as file:
    relevant_olive_oil_categories = file.read().splitlines()
    file.close()
olive_oil = read_and_filter_data('olive_oil.csv',relevant_olive_oil_categories)
olive_oil_origin = 'Greece' #Greece=6 is highest among all origins_tags


#user-defined function on sourdough.csv
with open("data/relevant_sourdough_categories.txt", "r") as file:
    relevant_sourdough_categories = file.read().splitlines()
    file.close() 
sourdough = read_and_filter_data('sourdough.csv',relevant_sourdough_categories)
sourdough_origin = 'United Kingdom' #United Kingdom=3 is highest among all origins_tags


relevant_salt_cat = [
 'en:edible-common-salt',
 'en:salts',
 'en:sea-salts',]
#user-defined function on salt.csv
salt_flakes = read_and_filter_data('salt_flakes.csv',relevant_salt_cat)

-----Avocado Origin-----
en:peru             2
en:spain,en:peru    1
en:chile,en:peru    1
en:israel           1
Name: origins_tags, dtype: int64
-----lemon Origins----- 
 en:brazil,en:south-africa    1
en:south-africa              1
Name: origins_tags, dtype: int64 

-----olive_oil Origins----- 
 en:greece                                             6
en:spain                                              4
en:italy                                              4
en:greece,en:italy,en:portugal,en:spain,en:tunisia    2
en:produce-of-italy                                   1
en:european-union-and-non-european-union              1
en:produced-in-italy                                  1
en:european-union                                     1
Name: origins_tags, dtype: int64 

-----sourdough Origins----- 
 en:united-kingdom    3
en:france            1
Name: origins_tags, dtype: int64 

-----salt_flakes Origins----- 
 Series([], Name: origins_tags, dtype: int64) 

